In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold

In [14]:
#for file_name in ["FINAL_BASE_DATASET_TEAM_RATINGS_WITH_NO_ZEROS_ADDED_NO_BETS_AGE_BMI_ZEROS_NOT_REMOVED_WITH_SEASON.csv", "FINAL_BASE_DATASET_WITH_BETS_WITH_SEASON.csv", "FINAL_BASE_DATASET_WITH_BETS_ZEROS_REMOVED_WITH_SEASON.csv", "FINAL_BASE_DATASET_WITH_SEASON.csv"]:
for file_name in ["FINAL_BASE_DATASET_WITH_BETS_ZEROS_REMOVED_WITH_SEASON.csv"]:
    data = pd.read_csv(file_name, index_col=0)
    
    #for season in ["2008/2009", "2009/2010", "2010/2011", "2011/2012", "2012/2013", "2013/2014", "2014/2015"]:
    data_train = data[data["season"] != "2015/2016"]
    #data_train = data[((data["season"] == "2014/2015")|(data["season"] == "2013/2014"))]
    data_test = data[data["season"] == "2015/2016"]

    X_train = data_train.drop("Label", axis = 1) # to fit the scaler
    X_train = X_train.drop("season", axis = 1)
    X_test = data_test.drop("Label", axis = 1)
    X_test = X_test.drop("season", axis = 1)
    y_train = data_train.Label
    y_test = data_test.Label

    max_val = 0

    for scaler in [RobustScaler(), StandardScaler(), MaxAbsScaler(), RobustScaler()]:
        scaler.fit(X_train)
        X_train_scaled = scaler.transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        for n_columns in range(1, X_train.shape[1] + 1):                   
            pca = PCA(n_components=n_columns).fit(X_train_scaled)
            X_train_pca = pca.transform(X_train_scaled)
            X_test_pca = pca.transform(X_test_scaled)

            clf = GaussianNB()
            clf.fit(X_train_pca, y_train)
            y_pred = clf.predict(X_test_pca)
            y_pred_proba = clf.predict_proba(X_test_pca)

            #print confusion_matrix(y_pred=y_pred_g, y_true=y_test)
            #print f1_score(y_test, y_pred_g, average='weighted')
            acc_score = accuracy_score(y_true=y_test, y_pred=y_pred)
            #print acc_score
            if acc_score > max_val:
                max_val = acc_score
            if acc_score > 0.50:
                print scaler, n_columns, acc_score
    print "%s, %f" % (file_name, max_val)

RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) 2 0.501813236627
RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) 3 0.501359927471
RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) 5 0.503173164098
RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) 7 0.504533091568
StandardScaler(copy=True, with_mean=True, with_std=True) 1 0.508159564823
StandardScaler(copy=True, with_mean=True, with_std=True) 2 0.503173164098
StandardScaler(copy=True, with_mean=True, with_std=True) 3 0.503626473255
StandardScaler(copy=True, with_mean=True, with_std=True) 4 0.504533091568
StandardScaler(copy=True, with_mean=True, with_std=True) 29 0.501813236627
RobustScaler(copy=True, quantile_range=(25.0, 75.0), with_centering=True,
       with_scaling=True) 2 0.501813236627
RobustScaler(copy=True, quantile_range=(25.0

In [2]:
data = pd.read_csv("./../Preprocessing/Data_Structures/FINAL_BASE_DATASET_WITH_BETS_WITH_SEASON.csv", index_col=0)

data_train = data[data["season"] != "2015/2016"]
data_test = data[data["season"] == "2015/2016"]

data.head()
y_train = data_train.Label
X_train = data_train.drop("Label", axis = 1)
X_train = X_train.drop("season", axis = 1)
y_test = data_test.Label
X_test = data_test.drop("Label", axis = 1)
X_test = X_test.drop("season", axis = 1)

scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

#FINAL_BASE_DATASET_WITH_SEASON
pca = PCA(n_components=1).fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

In [1223]:
#priors
#print y_train.value_counts() / len(y_train)
#proba

In [3]:
clf_g = GaussianNB()
clf_g.fit(X_train, y_train)
y_pred_g = clf_g.predict(X_test)
y_pred_proba_g = clf_g.predict_proba(X_test)

#clf_b = BernoulliNB()
#clf_b.fit(X_train, y_train)
#y_pred_b = clf_b.predict(X_test)
#y_pred_proba_b = clf_b.predict_proba(X_test)
#
#clf_m = MultinomialNB()
#clf_m.fit(X_train, y_train)
#y_pred_m = clf_m.predict(X_test)
#y_pred_proba_m = clf_m.predict_proba(X_test)

In [4]:
print "Gaussian NB"
print confusion_matrix(y_pred=y_pred_g, y_true=y_test)
print f1_score(y_test, y_pred_g, average='weighted')
print accuracy_score(y_true=y_test, y_pred=y_pred_g)

#print "Bernoulli NB"
#print confusion_matrix(y_pred=y_pred_b, y_true=y_test)
#print f1_score(y_test, y_pred_b, average='weighted')
#print accuracy_score(y_true=y_test, y_pred=y_pred_b)
#
#print "Multinomial NB"
#print confusion_matrix(y_pred=y_pred_m, y_true=y_test)
#print f1_score(y_test, y_pred_m, average='weighted')
#print accuracy_score(y_true=y_test, y_pred=y_pred_m)

Gaussian NB
[[420   0 251]
 [256   0 300]
 [276   0 703]]
0.436856960531
0.509066183137


/Users/steeve/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [1217]:
y_pred_proba

array([[ 0.25382247,  0.52077159,  0.22540594],
       [ 0.26512464,  0.32042593,  0.41444943],
       [ 0.01332757,  0.21797002,  0.76870241],
       ..., 
       [ 0.16059551,  0.31382556,  0.52557893],
       [ 0.61020988,  0.37611324,  0.01367688],
       [ 0.61789557,  0.31743048,  0.06467395]])

# Results
- 50.23% with Season 2014/15 as Training and 2015/16 as Prediction
FINAL_BASE_DATASET_WITH_BETS_ZEROS_REMOVED_WITH_SEASON, StandardScaler, PCA = 6
- 50.59% with Season 2013/14 and 2014/15 as Training and 2015/16 as Prediction
FINAL_BASE_DATASET_WITH_BETS_ZEROS_REMOVED_WITH_SEASON, StandardScaler, PCA = 3
- 50.81% with all Seasons before 2015 as Training and 2015/16 as Prediction
FINAL_BASE_DATASET_WITH_BETS_ZEROS_REMOVED_WITH_SEASON, StandardScaler, PCA = 1